---


---



---




---



---



---



---



---



---

### FLASK

In [1]:
from flask import Flask, request, jsonify
import torch
from torch import nn
from transformers import BertModel, BertTokenizer
import os

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
app = Flask(__name__)

In [3]:
# Configuración del modelo y el tokenizador
PRE_TRAINED_MODEL_NAME = 'ignacio-ave/BETO-nlp-sentiment-analysis-spanish'
path_checkpoint = "Checkpoint/modelo_BETO_ultimo_checkpoint.pth"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

category_mapping = {
    '0': 0,
    '1': 1,
}

In [4]:
device

device(type='cuda', index=0)

In [5]:
class BETOMusicForKidsClassifier(nn.Module):
    def __init__(self, n_classes):
        super(BETOMusicForKidsClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.linear = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
                            input_ids=input_ids,
                            attention_mask=attention_mask
                            )
        pooled_output = outputs.pooler_output
        drop_output = self.drop(pooled_output)
        output = self.linear(drop_output)
        return output

In [6]:

model = torch.load(path_checkpoint)

In [7]:

tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [8]:
def predict_genre(text, model, tokenizer, device, max_length=200):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, predicted_class = torch.max(outputs, dim=1)

    predicted_genre = list(category_mapping.keys())[list(category_mapping.values()).index(predicted_class.item())]
    return predicted_genre

In [9]:
@app.route('/classify2', methods=['POST'])
def classify2():
    data = request.json
    if 'text' not in data:
        return jsonify({'error': 'No text provided'}), 400

    text = data['text']
    genre = predict_genre(text, model, tokenizer, device)
    return jsonify({'genre': genre})

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\LENOVO LOQ\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
